# MultiTable

## Set up source relational data

In [ ]:
# Display the schema of our demo database

from IPython.display import Image

Image("https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecommerce_db.png", width=600, height=600)

In [ ]:
# Download the demo database

!curl -o "ecom_xf.db" "https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecom_xf.db"

In [ ]:
# Connect to SQLite database and extract relational data

from gretel_trainer.relational.connectors import SQLite

ecommerce_db_path = "sqlite:///ecom_xf.db"

sqlite = SQLite(db_path=ecommerce_db_path)
relational_data = sqlite.extract()

In [ ]:
# Alternatively, manually define relational_data

from gretel_trainer.relational.core import RelationalData
import pandas as pd

csv_dir = "/path/to/extracted_csvs"

rel_data = RelationalData()

rel_data.add_table("events", "id", pd.read_csv(f"{csv_dir}/events.csv"))
rel_data.add_table("users", "id", pd.read_csv(f"{csv_dir}/users.csv"))
rel_data.add_table("inventory_items", "id", pd.read_csv(f"{csv_dir}/inventory_items.csv"))
rel_data.add_table("products", "id", pd.read_csv(f"{csv_dir}/products.csv"))
rel_data.add_table("distribution_center", "id", pd.read_csv(f"{csv_dir}/distribution_center.csv"))
rel_data.add_table("order_items", "id", pd.read_csv(f"{csv_dir}/order_items.csv"))

rel_data.add_foreign_key("events.user_id", "users.id")
rel_data.add_foreign_key("order_items.user_id", "users.id")
rel_data.add_foreign_key("order_items.inventory_item_id", "inventory_items.id")
rel_data.add_foreign_key("inventory_items.product_id", "products.id")
rel_data.add_foreign_key("inventory_items.product_distribution_center_id", "distribution_center.id")
rel_data.add_foreign_key("products.distribution_center_id", "distribution_center.id")


## Do things with the source data!

In [ ]:
from gretel_trainer.relational.multi_table import MultiTable

multitable = MultiTable(relational_data=relational_data, project_prefix="mtnb")

In [ ]:
# Transform some tables

xform_out = multitable.transform(
    configs={
        "users": "https://gretel-blueprints-pub.s3.amazonaws.com/rdb/users_policy.yaml",
        "events": "https://gretel-blueprints-pub.s3.amazonaws.com/rdb/events_policy.yaml",
    }, 
    in_place=False
)

In [ ]:
# Compare original to transformed

print(multitable.relational_data.get_table_data("users").head(5))
print(xform_out["users"].head(5))

In [ ]:
# Train synthetic models for all tables

multitable.train()

In [ ]:
# Inspect training results

multitable.train_statuses

The `products` and `inventory_items` tables from the ecommerce dataset include columns containing only null / None / NaN values, which consistently lead to ACTGAN training failures. The next cell demonstrates how to provide a modified versions of tables and retrain _just those tables_ without throwing away and retraining the other tables that trained successfully. (For demo simplicity, we'll simply drop the problematic columns.)

In [ ]:
original_products = multitable.relational_data.get_table_data("products")
original_inventory_items = multitable.relational_data.get_table_data("inventory_items")

modified_products = original_products.drop(columns=["sku"])
modified_inventory_items = original_inventory_items.drop(columns=["product_sku"])

multitable.retrain_tables({
    "products": modified_products,
    "inventory_items": modified_inventory_items,
})

In [ ]:
# Generate synthetic data

synthetic_tables = multitable.generate()

In [ ]:
# Inspect generation results

multitable.generate_statuses

In [ ]:
# Write output data to a new SQLite database

from gretel_trainer.relational.connectors import SQLite

out_db_path = "sqlite:///out.db"

out_db = SQLite(db_path=out_db_path)
out_db.save(synthetic_tables, prefix="syn_")

In [ ]:
# Cleanup / reset

import gretel_client

projects = gretel_client.projects.projects.search_projects(query="mtnb") 
for project in projects:
    project.delete()

!rm -rf working